# Sardana Overall Architecture and General Concepts

## [Symbolic sketch](http://www.sardana-controls.org/en/latest/users/overview.html#symbolic-sketch)

## Contents

* Sardana plugins
* Pool core
* MacroServer core
* Concurrency
* Tango layer
* Taurus: Qt and Core

## Sardana plugins

* Entry points
 * controllers
 * macros
 * recorders
 * macro parameters editors (beta)
 * Sardana-Taurus extensions are plugins to Taurus
* Discovery methods
* Plugin system architecture
* To be unified and formalized by [TEP13](https://github.com/taurus-org/taurus/blob/develop/doc/source/tep/TEP13.md)

### Discovery methods

* Path based. Scannning the directory, not subdirectories.
* Uses `inspect` module and `issubclass` function

* Configurable with Tango properties:
 * `PoolPath`, `MacroPath`, `RecorderPath`
 * Order precedence
 * Paths starting with `#` are ignored

* Default paths: 
 * Controllers: `sardana/pool/poolcontrollers`
 * Macros: `sardana/macroserver/macros`
 * Recorders: `sardana/macroserver/recorders`

### Plugin system architecture

TODO: class diagram

Manager e.g. ControllerManager, MacroManager, ...
* (Re)loads modules and classes using inspect
* Maintains register of present modules and classes
* Creates new plugins from template

### Plugin system architecture

Meta classes e.g. ControllerClass, MacroLibrary:
* represents plugin module or class
* has reference to the plugin (class or module)
* contains meta information
 * generic e.g. path, file name, etc.
 * specific e.g. macro parameters, pseudo motor roles, etc.

### Plugin system architecture

Plugin classes and modules:
* classes - subclasses of `Controller`, `Macro` or `Recorder`
* modules - python modules that contains plugin classes

### Plugin system architecture

Plugin instances:
* Instances of plugin classes e.g. IcePAP controller, ascan macro or SPEC recorder

## Pool core

* TODO: class diagram
* Value, Attribute, Buffer and Event
* Elements e.g. Motor, CT
* Controllers
* Containers e.g. MotorGroup, PseudoMotor
* Pseudo elements e.g. PseudoMotor, PseudoCounter
* Instruments

### Element

* Has name, full name and unique id within the Pool
* Has reference to its controller
* Identifiable by axis number (`int`) in the controller
* Has state and status
* Generates or forwards events
* May have attributes and parameters
* Has a dedicated action object e.g. `PoolMotion` for a `PoolMotor` element
* May be stopped or aborted
* May be associated to an instrument
* Is serializable

### Value, Attribute, Buffer and Event

Value e.g. Motor's position readout:
* Value
* Timestamp
* Error

### Value, Attribute, Buffer and Event

Event:
* Publisher-Subscriber design pattern
* EventGenerator has references (`weakref`) to its listeners (callbacks)
* Emitting an event means the callback execution in the synchronous way
* Callback arguments: source, type and value
* EventType has name and priority
* Example: Offset -> Position

### Value, Attribute, Buffer and Event

Attribute e.g. PseudoCounter's value:
* Has read & write value
* Has configuration e.g. limits, alarms... (not used at the moment)
* Has reference to its element

### Value, Attribute, Buffer and Event

Buffer e.g. CounterTimer's value buffer:
* Used by experimental channels
* Works with ordered and indexed values
* May buffer values if pseudo counters require that
* Works by appending or extending the buffer
* Has reference to its element

### Controller

* Container of elements
* Has reference to the controller plugin instance
* May have attributes and parameters
* Has interface to access element's attributes and parameters
* Implements raw algorithms to:
 * State and Value access
 * Move, Stop and Abort commands
* PseudoMotor and PseudoCounter controllers are specializations that provides interface to calculations
* Has its own log level

### Element Container

* Type: MotorGroup, MeasurementGroup, PseudoMotor, PseudoCounter
* Has references to its physical elements
* Its state is composed from states of physical elements
* Stop and abort acts on all physical elemenets
* Has a dedicated action object e.g. `PoolMotion` for a `PoolPseudoMotor` element

### Pseudo Element

* Types: PseudoMotor, PsedoCounter
* Inherits behavior of Element Container
* Has references to its siblings
* Its default attribute i.e. Position or Value listens to the physical attributes and forwards events
* TODO: show on the class diagram how events are propagated

### Instrument

* Parent <-> Children relations may be defined
* "Lighweight" element e.g. no state, no attributes, etc.
* Any element may be assigned to an instrument
* Managed by Pool
* Nexus recorder uses this information to provide links to the measurement data in an organized way
* TaurusGUI uses this information to populate one panel per instrument and link it with the synoptic

## MacroServer

* Runs *sequences* i.e. macros, scripts on a Door, also concurrently
* Contains catalogue of available macros
* MacroServer can connect to multiple Pools but can also run independently
* Core concepts:
 * Macros
 * Environment
 * Hooks
 * Generic Scan Framework (GSF)

### Macros

* Python scripts i.e. class or function
* Are created and destroyed on execution
* The last run macro may be kept in memory to allow access to its data
* Usually a single thread of execution
* Are executed asynchronously by the MacroExecutor

### Environment
* Environment to access shared data (similar to bash environment)
* Multiple levels: global, door, macro
* Macros may require a given environment variable to be defined
* Persistency achieved with `shelve` file
* Configurable with `EnvironmentDb` - be careful with the default location - `/tmp`
* Taurus scheme is pending to be implemented - see [SEP14](https://github.com/sardana-org/sardana/blob/develop/doc/source/sep/SEP14.md)
* Environment inheritance between wrapper and nested macros is missing

In [6]:
import shelve
env = shelve.open("/home/sicilia/workspace/macroserver.properties")
print env["ActiveMntGrp"]
env.close()

mntgrp01


### Hooks

* Code that is executed at a given moment within a macro
* May be another macro or a python callable
* A macro may define allowed hook places - where the hooks may be attached
* Example: scans defines hook places like: pre-scan, pre-acq, post-move, etc.
* Hook may be attached programatically or using `sequencer`
* General Hooks implementaion is pending - see [#200](https://github.com/sardana-org/sardana/issues/200)

### GSF

* Control Moveables (if involved) and MeasurementGroup to perfom a scan
* Step, Hybrid and Continuous (`ascanc` & `ascanct`) modes
* Customizable with generators e.g. step, period, waypoint generators
* Customizable with hooks
* Parametrizable with environment variables
* Executes pre-scan snapshot
* Handles data using recorders
* Estimates duration
* Provides statistics e.g. dead time, motion time, etc.
* Reports progress
* Provides a set of multidimensional scan macros

## Elements attribute

* Both Pool and MacroServer defines it
* Each elemenent is serialized and represented in Elements attribute
* Events notifies about `new`, `del` and `change` elements
* MacroServer listens to the Elements events comming from its Pools
* Is JSON encoded
* Pool and MacroServer restart orer is important due to [#113](https://github.com/sardana-org/sardana/issues/113)

## Worker threads and jobs

* Concurrency achieved using threads
* `taurus.core.util.threadpool` is used
* Pool thread pool is used to:
 * run actions: motion, acquisition, synchronization
 * access controllers: read State and Value
* MacroServer thread pool is used to:
 * run macros
 * stop/abort reserved objects
 * run concurent tasks in GSF e.g. go through waypoints
 * handle Data (in the future will be renamed to ValueBuffer) events in GSF (continuous scan)
* Tango layer uses pool thread to push events

## Tango layer

* Currently the only way to run Sardana sercerss is by using Tango
* Defines device classes and device servers
* Use events to:
 * notify clients about attribute updates (with change criteria)
 * transfer data in continuous scan 
* Extensively uses dynamic attributes
* Attribute ranges are used as software limits (it has limitations: [#9](https://github.com/sardana-org/sardana/issues/9), [#159](https://github.com/sardana-org/sardana/issues/159), [#36](https://github.com/sardana-org/sardana/issues/36) and [#259](https://github.com/sardana-org/sardana/issues/259))
* Persistance is achieved using Tango DB:
 * device servers definition (device names and aliases)
 * device and attribute properties
 * memorized attributes
* Uses Tango monitor as the serialization mechanism
* Neither Tango polling nor Tango logging is used 

## Taurus

* Qt: GUI, widgets
* Core
 * Tango scheme extensions
 * MSenv scheme ([SEP14](https://github.com/sardana-org/sardana/blob/develop/doc/source/sep/SEP14.md))
 * utils

### Taurus Qt

* TaurusGUI
 * connection to MacroServer and Door
 * Instrument panels and its connection with synoptic (optional)
 * General stop button
 * Example `taurusgui macrogui`
* Pool elements widgets:
 * Motor
 * ExpChannel
 * IORegister

### Taurus Qt

* Macro execution widgets:
 * macroexecutor
 * sequencer
 * macrobutton
 * door streams e.g. output, debug...
* Experiment configuration (spock command: `expconf`)
 * online plot
* Offline plot (spock command: `showscan`)
* Sardana editor

### Taurus Core - Tango scheme extension

* Each Tango class has its extension equivalent e.g. Motor, MeasurementGroup, etc.
* MeasurementGroup configuration attribute has a helper class (it is not extension)
* ExperimentConfiguration helper class (MeasurementGroup configurations + environment variables e.g. PreScanSnapshot, ScanDir, ...)
* Pool elements allows synchronous actions e.g. motion, acquisition based on Start command and State events
* MacroServer allows sychronous macro execution e.g. used by spock
* Provides event handling decoupled from Tango see: `umv` macro as an example
* MacroServer provides macro node classes used as model in macroexecution widgets

In [3]:
import taurus
from sardana.taurus.core.tango.sardana import registerExtensions, unregisterExtensions

registerExtensions()
mot = taurus.Device("mot01")
print type(mot)
unregisterExtensions()  # this does not clean the Taurus factory

<class 'sardana.taurus.core.tango.sardana.pool.Motor'>


### Taurus Qt Core - Tango scheme extension

* Similar to taurus core extension
* Translate Tango events to Qt signals
* Foreseen to be used in Qt widgets

In [4]:
import taurus
from sardana.taurus.qt.qtcore.tango.sardana import registerExtensions

registerExtensions()
door = taurus.Device("Door_zreszela_1")
print type(door)

<class 'sardana.taurus.qt.qtcore.tango.sardana.macroserver.QDoor'>
